In [140]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.sql.functions import col, log

In [141]:
spark = SparkSession.builder.appName('Scores').getOrCreate()

dataset = spark.read.option("inferSchema", "true").csv("Player_List_Cleaned_Data.csv", header=True)

dataset.printSchema()

root
 |-- Player: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- Overall_Score: double (nullable = true)
 |-- Potential_Score: double (nullable = true)
 |-- Market_Value: double (nullable = true)
 |-- Weekly_Salary: double (nullable = true)
 |-- Height: double (nullable = true)
 |-- Weight: double (nullable = true)
 |-- Age: double (nullable = true)
 |-- Preferred_Foot: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Player_Work_Rate: string (nullable = true)
 |-- Kit_Number: double (nullable = true)
 |-- Joined_Club: string (nullable = true)
 |-- Contract_Length: double (nullable = true)
 |-- Ball_Skills: double (nullable = true)
 |-- Defence: double (nullable = true)
 |-- Mental: double (nullable = true)
 |-- Passing: double (nullable = true)
 |-- Physical: double (nullable = true)
 |-- Shooting: double (nullable = true)
 |-- Goalkeeping: double (nullable = true)
 |-- Description_Player: string (nullable 

In [142]:
dataset.show()

+--------------------+--------------+-------------------+-------------+---------------+------------+-------------+------+------+----+--------------+--------------+----------------+----------+--------------+---------------+-----------+------------------+------------------+------------------+-----------------+--------+-----------+--------------------+
|              Player|       Country|               Club|Overall_Score|Potential_Score|Market_Value|Weekly_Salary|Height|Weight| Age|Preferred_Foot|      Position|Player_Work_Rate|Kit_Number|   Joined_Club|Contract_Length|Ball_Skills|           Defence|            Mental|           Passing|         Physical|Shooting|Goalkeeping|  Description_Player|
+--------------------+--------------+-------------------+-------------+---------------+------------+-------------+------+------+----+--------------+--------------+----------------+----------+--------------+---------------+-----------+------------------+------------------+------------------+-----

In [143]:
list_col = ['Market_Value' ,'Player', 'Country', 'Club', 'Overall_Score',
            'Potential_Score', 'Weekly_Salary', 'Height', 'Weight',
            'Preferred_Foot', 'Position', 'Player_Work_Rate', 'Kit_Number',
            'Joined_Club', 'Contract_Length', 'Ball_Skills', 'Mental','Description_Player']

dataset = dataset[dataset['Market_Value'] != 1000]
dataset = dataset.withColumn("Market_Value_Log", log(col("Market_Value")))
dataset = dataset.drop(*list_col)
dataset.show(20)

+----+------------------+------------------+-----------------+--------+-----------+------------------+
| Age|           Defence|           Passing|         Physical|Shooting|Goalkeeping|  Market_Value_Log|
+----+------------------+------------------+-----------------+--------+-----------+------------------+
|34.0| 26.33333333333333|              89.0|80.85714285714286|  86.875|       10.8|18.172219384653864|
|33.0|              32.0| 75.33333333333333|80.28571428571429|    88.0|       10.2| 18.59882692933584|
|23.0| 30.66666666666667|              78.0|87.42857142857143|    80.5|        8.4|19.083368717027604|
|28.0|              19.0|              32.0|59.42857142857143|    18.5|       87.4|18.534009429259367|
|30.0|              62.0| 93.33333333333331|76.28571428571429|  81.375|       11.2| 18.64781631653611|
|29.0|              32.0|              84.0|             80.0|  82.625|       11.8|18.675322962325946|
|36.0| 26.66666666666667| 81.33333333333333|             83.0|   88.25|  

In [144]:
dataset.describe().show()

+-------+------------------+------------------+-----------------+------------------+------------------+-----------------+------------------+
|summary|               Age|           Defence|          Passing|          Physical|          Shooting|      Goalkeeping|  Market_Value_Log|
+-------+------------------+------------------+-----------------+------------------+------------------+-----------------+------------------+
|  count|             18489|             18489|            18489|             18489|             18489|            18489|             18489|
|   mean|25.634214938612146| 47.21771503777027|54.16207835289443| 64.44685256870905| 48.01286575801828|16.28146465465954|13.897814814212955|
| stddev| 4.729868608623024|20.359235083089224|14.70088090841095|10.039973292300827|15.041247252067105|17.02011723945282|1.2461432331046034|
|    min|              17.0|               7.0|              8.0| 25.42857142857143|            11.375|              2.0| 7.600902459542082|
|    max|    

In [145]:
assembler = VectorAssembler(inputCols=['Age', 'Defence',
                                        'Passing', 'Physical',
                                        'Shooting', 'Goalkeeping'], outputCol='features')

data_set = assembler.transform(dataset)
data_set.select(['features', 'Market_Value_Log']).show(20)

+--------------------+------------------+
|            features|  Market_Value_Log|
+--------------------+------------------+
|[34.0,26.33333333...|18.172219384653864|
|[33.0,32.0,75.333...| 18.59882692933584|
|[23.0,30.66666666...|19.083368717027604|
|[28.0,19.0,32.0,5...|18.534009429259367|
|[30.0,62.0,93.333...| 18.64781631653611|
|[29.0,32.0,84.0,8...|18.675322962325946|
|[36.0,26.66666666...|17.622173047734595|
|[28.0,41.33333333...|18.679191439103867|
|[22.0,16.66666666...| 18.59882692933584|
|[29.0,16.66666666...|18.222229805228526|
|[26.0,83.0,89.333...|18.497641785088494|
|[28.0,17.33333333...|18.358805340234277|
|[29.0,40.66666666...|18.430631074805532|
|[30.0,90.0,72.666...|18.269857854217783|
|[29.0,86.66666666...| 18.29284737244248|
|[29.0,18.0,27.333...| 18.26402693390699|
|[24.0,88.0,70.666...|18.529535148864447|
|[21.0,39.0,58.333...| 18.78184559316395|
|[27.0,69.0,89.0,7...| 18.49300140553199|
|[27.0,89.33333333...|18.455082170669698|
+--------------------+------------

In [128]:
train_data, test_data = data_set.randomSplit([0.8, 0.2], seed=100)

In [146]:
train_data.show()

+----+------------------+-----------------+-----------------+--------+-----------+------------------+--------------------+
| Age|           Defence|          Passing|         Physical|Shooting|Goalkeeping|  Market_Value_Log|            features|
+----+------------------+-----------------+-----------------+--------+-----------+------------------+--------------------+
|17.0|              17.0|44.33333333333334|65.42857142857143|    57.5|        9.2|14.403297222866392|[17.0,17.0,44.333...|
|17.0|              20.0|54.33333333333334|64.57142857142857|    57.5|        7.0| 14.77102200299171|[17.0,20.0,54.333...|
|17.0|33.666666666666664|             65.0|69.57142857142857|  51.125|       11.4|14.648419680899378|[17.0,33.66666666...|
|17.0|              38.0|63.33333333333334|63.14285714285714|   55.25|       11.8| 14.28551418721001|[17.0,38.0,63.333...|
|17.0| 62.33333333333334|             64.0|             69.0|  53.875|       11.4|14.457364444136669|[17.0,62.33333333...|
|17.0| 63.666666

In [147]:
test_data.show()

+----+------------------+-----------------+-----------------+--------+-----------+------------------+--------------------+
| Age|           Defence|          Passing|         Physical|Shooting|Goalkeeping|  Market_Value_Log|            features|
+----+------------------+-----------------+-----------------+--------+-----------+------------------+--------------------+
|17.0|              35.0|57.33333333333334|66.28571428571429|  55.875|       11.8|14.403297222866392|[17.0,35.0,57.333...|
|18.0|14.666666666666664|             63.0|             74.0|  60.375|       10.0|15.520258650202699|[18.0,14.66666666...|
|18.0| 18.33333333333333|             45.0|68.42857142857143|  59.125|        9.6|14.457364444136669|[18.0,18.33333333...|
|18.0|              27.0|61.66666666666666|59.71428571428572|  64.875|        8.2|14.346138809026444|[18.0,27.0,61.666...|
|18.0|              30.0|70.33333333333333|68.57142857142857|   58.25|        8.4| 16.90655301132259|[18.0,30.0,70.333...|
|18.0|32.3333333

In [148]:
lr = LinearRegression(featuresCol = 'features', labelCol='Market_Value_Log')
lrModel = lr.fit(train_data)
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

Coefficients: [-0.09325732054107534,0.02511247795708845,0.02739503186411397,0.038104748246601344,0.054721387936536255,0.07687178301981262]
Intercept: 7.289264732807623


In [149]:
test_stats = lrModel.evaluate(test_data)
print(f"RMSE: {test_stats.rootMeanSquaredError}")
print(f"R2_Score: {test_stats.r2}")
print(f"meanSquaredError: {test_stats.meanSquaredError}")

RMSE: 0.8404122635006774
R2_Score: 0.5542589596149772
meanSquaredError: 0.7062927726423321


In [150]:
predict_data = lrModel.transform(test_data)
selected = predict_data.select('Market_Value_Log', 'prediction')
selected.show()

+------------------+------------------+
|  Market_Value_Log|        prediction|
+------------------+------------------+
|14.403297222866392|14.643920551441013|
|15.520258650202699| 14.59710931099308|
|14.457364444136669|13.884629492064882|
|14.346138809026444|14.433826653210417|
| 16.90655301132259| 14.73693205685069|
|14.845129975145433|14.429175896238075|
|15.520258650202699|15.274103845163141|
|14.845129975145433|14.979035217120881|
|14.603967918328545|14.751357234851922|
| 14.22097566607244|14.757830749619787|
| 14.22097566607244|14.881626125446648|
|17.610999747136468|16.084056848851635|
|14.690979295318174|14.167584841896371|
|14.457364444136669|13.837670823873848|
|14.346138809026444|14.289064764421529|
|15.068273526459642|14.899939288107493|
|15.009433026436708| 14.19758738119487|
|14.151982794585487|13.790822032537571|
|14.880221294956703|14.135460573776296|
|15.009433026436708|14.319523411898611|
+------------------+------------------+
only showing top 20 rows

